This post is one of a series concerning building a neural network from scratch. If anything confuses you I recommend starting at the beginning. First we will import our dependencies and load up the dataset that we have been working with in past posts. This is a big block of code but it is explained line by line in previous posts if it is confusing.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2, threshold=np.nan)

np.random.seed(42)  #preserve seed for replication

train_data_subset = np.loadtxt('train_data_subset.txt')        #load saved data arrays from post 3
train_label_subset = np.loadtxt('train_label_subset.txt', dtype=int)

# Model Parameters

hidden_layer_size = 3 #number of neurons in hidden layer
subset_sample_size = len(train_data_subset)
input_dimensions = 2 #x, y
output_dimensions = 2 #red, blue
learning_rate = 0.01 #rdm pick, should be experimented with in actual model
parameters = {}    #define dictionary to hold weights/biases
W1 = np.random.randn(input_dimensions, hidden_layer_size)*(1/np.sqrt(input_dimensions)) # 2x3 rdm scaled matrix
b1 = np.zeros((1, hidden_layer_size)) #zeros bias matrix 1x3
W2 = np.random.randn(hidden_layer_size, output_dimensions)*(1/np.sqrt(input_dimensions))# 3x2 rdm scaled matrix
b2 = np.zeros((1, output_dimensions)) #zeros bias matrix 1x2
parameters = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2} # update dictionary with keys and values

# Review Feed-Forward Math and Create a Computation Graph

Now we will quickly review and define our feed forward calculations. With $x$ as the input, we calculate the following:

$$
\begin{align*}
&1. \quad x \cdot W_1 = i_1 \\
&2. \quad i_1 + b_1 = a_1 \\
&3. \quad \tanh(a_1) = z_1 \\
&4. \quad z_1 \cdot W_2 = i_2 \\
&5. \quad i_2 + b_2 = a_2 \\
&6. \quad \textrm{softmax}(a_2) = y \\
&7.  \quad \textrm{cross_entropy}(y) = \textrm{cost}
\end{align*}
$$

A handy way to view this is to make a computation chart. This chart will have the weights, biases, and x as inputs and will prove valuable when calculating derivatives later.

![Reverse_Auto-diff](http://codeouts.com/wp-content/uploads/2018/02/chart-1.png)

Now we can code out these feed forward steps in python. Note: we will save the last cross-entropy step for later and go into more detail with it.

In [2]:
i1 = train_data_subset.dot(W1) 
a1 = i1 + b1
z1 = np.tanh(a1)
i2 = z1.dot(W2)
a2 = i2 + b2

softmax = np.exp(a2) / np.sum(np.exp(a2), axis=1, keepdims=True)

answers = np.argmax(softmax, axis=1) #predicted class

print(softmax)
print(train_label_subset)
print(answers)

[[0.62 0.38]
 [0.54 0.46]
 [0.55 0.45]
 [0.66 0.34]
 [0.66 0.34]
 [0.66 0.34]
 [0.48 0.52]
 [0.55 0.45]]
[0 1 1 0 0 1 0 1]
[0 0 0 0 0 0 1 0]


# The Cross-Entropy Cost Function

The numbers populating the array above are the outputs of randomly generated weights after 1 round of forward propagation in our single hidden layer neural network. The numbers represent the probability of the coordinates being of the blue or red class. The 1x8 array below the softmax array contains the correct classes for our data and the last 1x8 array are the predicted classes.

For example, the first number in the label array is a zero meaning that the point is red. In our softmax array, the probability of the point being zero (red) is given as 62% and being one (blue) as 38%. In this case, our model correctly predicted the class of the point! We shouldn't be excited, however, as if we look down the list we see that it is only correct about half the time- which makes sense as the numbers were essentially randomly generated. 

What we want to do now is find a way to let the model know if it is on the right track. We want to reward correct predictions and/or punish incorrect predictions. 

A good cost function for neural networks must satisfy two general requirements:

1. The Cost Function $C$ must be able to be described as an average $C= \frac{1}{n} \sum C_s$ of cost functions $C_s$ for $s$ individual training outputs

2. The Cost Function $C$ must only be dependant upon the output values of the last layer (in our case $z_2$ scaled with softmax). This allows backpropagation later.

The cross-entropy cost function works well for this! In this case we can define the function as: 

$$
C = - \frac{1}{n} \sum_{n} \hat{y_n} \log y_i
$$

Where $n$ is the size of the input dataset ($x$). Remember $\hat{y}$ is a one hot encoded vector with the correct answer. In this case, it will always be 1. In python our cross-entropy cost function looks like this:

In [3]:
desired_softmax =  -np.log(softmax[range(subset_sample_size), train_label_subset]) 
cost = (-1 / subset_sample_size) * (np.sum(desired_softmax))

print(cost)

-0.6890016217265902


# Reverse  Auto-Differentiation

In order to train our weights and biases we need to define how small changes in each weight and bias affect the final cost. To do this we can use a form of the chain rule in order to find the derivative of the cost function with respect to each weight and bias. I suggest copying down the computation graph from above as this will make it easier to follow each step. 

What we will do first is calculate the derivative of each node with respect to each node that contributes an input to it.

The derivative of the Cost function with respect to the softmax function is $\frac{\partial{C}}{\partial{\sigma}} = y - \hat{y}$. There is a full derivation of this at the end of this post which will help if you haven't seen this derivative before. 

Here is a list of each derivative going down the chart:

$$
\begin{align*}
&7. \quad \frac{\partial{C}}{\partial{\sigma}} = y - \hat{y} 
&&6.\quad \frac{\partial{\sigma}}{\partial{a_2}} = 1 \\
&5. \quad \frac{\partial{a_2}}{\partial{b_2}} = 1 
&&5. \quad \frac{\partial{a_2}}{\partial{i_2}} = 1 \\
&4. \quad \frac{\partial{i_2}}{\partial{z_1}} = W_2^\intercal 
&&4. \quad \frac{\partial{i_2}}{\partial{W_2}} = z_1^\intercal \\
&3. \quad \frac{\partial{z_1}}{\partial{a_1}} = (1 - \tanh^2 a_1) 
&&2. \quad \frac{\partial{a_1}}{\partial{i_1}} = 1 \\
&2. \quad \frac{\partial{a_1}}{\partial{b_1}} = 1 
&&1. \quad \frac{\partial{i_1}}{\partial{W_1}} = x^\intercal \\
&1. \quad \frac{\partial{i_1}}{\partial{x}} = W_1^\intercal
\end{align*}
$$

Now we can combine these results (following the flow of the graph) in order to find the derivative of the cost function with respect to each of the weights and biases. We won't bother with the input x as that is static and can't be changed.

$$
\begin{align*}
&\frac{\partial{C}}{\partial{W_2}} = \quad (z_1^\intercal ) (1) (1)(y - \hat{y}) = \quad z_1^\intercal \cdot (y-\hat{y}) \\
&\frac{\partial{C}}{\partial{b_2}} = \quad (y - \hat{y}) \\
&\frac{\partial{C}}{\partial{b_1}} = \quad (1 - \tanh^2 a_1) * ((W_2^\intercal) \cdot(y-\hat{y})) \\
&\frac{\partial{C}}{\partial{W_1}} = \quad (x^\intercal) \cdot ((1-\tanh^2 a_1) *((W_2^\intercal) \cdot (y-\hat{y})))
\end{align*}
$$



# Backpropagation 

Awesome this is the last of the math that we need to back propagate through our model. The following implements the above results in python:

In [4]:
dC = softmax
dC[range(subset_sample_size), train_label_subset] -= 1

db2 = np.sum(dC, axis=0, keepdims=True)
dW2 = (z1.T).dot(dC)
delta = dC.dot(W2.T) * (1- np.power(a1, 2))
db1 = np.sum(delta, axis=0, keepdims=True)
dW1 = np.dot(train_data_subset.T, delta)

print(dW1)
print(db1)

[[ 7.00e-01 -1.68e+00  1.70e-03]
 [ 1.66e-01  5.32e-01 -1.38e-03]]
[[ 6.66e-01 -5.09e-01  9.67e-05]]


And that is it! These are all the derivatives (gradients) that we will need to backpropagate through our model. Now we just need to write a bit of code to update our weights and biases according to these gradients:

In [5]:
W1 = W1 + (-learning_rate * dW1)
b1 = b1 + (-learning_rate * db1) 
W2 = W2 + (-learning_rate * dW2)
b2 = b2 + (-learning_rate * db2)

parameters = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2} # update dictionary with keys and values

Now our model is able to adjust the weights and biases according to feedback received from the output. This will allow the model to be trained and optomized for specific datasets.

# Math Addendum - Derivative of Cost Function with respect to Softmax

First we need to find the derivative of the softmax function we used to output probabilities at the end of forward propagation.
$$ 
\frac{\partial{\sigma_j}}{\partial{z}_j} = \frac{\partial{}}{\partial{z}_j}\frac {e^{z_i}}{\sum_{n=1}^{N} e^{z_n}}\\
$$

apply the quotient rule (Note that $\frac{\partial{}}{\partial{z}_j}{e^{z_i}}{\sum_{n=1}^{N} e^{z_n}} = e^{z_j}$ - if this makes no sense I have included a vector calculus review for this at the bottom).

If $i =j$:

$$
= \frac{ e^{z_i} {\sum_{n=1}^{N} e^{z_n}} - e^{z_j} e^{z_i} } {({\sum_{n=1}^{N} e^{z_n}})^2} \\
\\
=  \frac{ e^{z_i} ({\sum_{n=1}^{N} e^{z_n}} - e^{z_j}) } {({\sum_{n=1}^{N} e^{z_n}})^2} \\
\\
= \frac {e^{z_i}}{\sum_{n=1}^{N} e^{z_n}} * \frac{{\sum_{n=1}^{N} e^{z_n}} - e^{z_j}} {({\sum_{n=1}^{N} e^{z_n}})^2} \\
\\
= \sigma (1 - \sigma)
$$

and if $i \neq j$:

$$
= \frac{ 0 - e^{z_j} e^{z_i} } {({\sum_{n=1}^{N} e^{z_n}})^2}  \\
\\
=  \frac{ - e^{z_j}} {{\sum_{n=1}^{N} e^{z_n}}} * \frac{ e^{z_i}} {{\sum_{n=1}^{N} e^{z_n}}} \\
\\
= - \sigma_j * \sigma_i
$$ 

So we know now that 
$$
\frac{\partial{\sigma_j}}{\partial{z}_j} = \sigma (1 - \sigma) \quad \textrm{if} \quad i = j \\
\\
\quad  \textrm{and} \\
\\
\frac{\partial{\sigma_j}}{\partial{z}_j} = - \sigma_j * \sigma_i \quad \textrm{if} \quad i \neq j
$$


Ok now for the derivative of our cross entropy cost function with respect to the softmax derivative. 

$$
C = - \sum_{n} \hat{y_n} \log{(y_n)} \\
\\
\frac{\partial{C}}{\partial{\sigma_n}} = - \sum_{i} \hat{y_i} \frac{\partial{\log{y_i}}}{\partial{\sigma_n}} \\
\\
= - \sum_{i} \hat{y_i} \frac{\partial{\log{y_i}}}{\partial{y_i}} * \frac{\partial{y_i}}{\partial{\sigma_i}} \\
\\
= - \sum_{i} \hat{y_i} \frac{1}{y_i} * \frac{\partial{y_i}}{\partial{\sigma_n}}
$$

which we can plug the eariler softmax equations into:

$$
    \frac{\partial {C}}{\partial{\sigma_n}} = - \hat{y_n}(1-y_n) - \sum_{ n \neq i} \hat{y_i} \frac {1}{y_n}(-y_i * y_n) \\
\\
= - \hat{y_n} +y_n \hat{y_n} + \sum_{i \neq 1}  \hat{y_i}y_n \\
\\
= y_n ( \hat{y_n} + \sum_{i \neq 1} \hat{y_i}) - \hat{y_n}
$$

Remember that $ \hat{y} $ is a one hot encoded vector (has a value of either 0 or 1) and thus $\hat{y_n} + \sum_{i \neq 1} \hat{y_i} = 1$ . Thus:

$$
\frac{\partial {C}}{\partial{\sigma_n}} = y_n - \hat{y_n}
$$

And finally a quick vector calculus review to show that $\frac{\partial{}}{\partial{z}_j}{e^{z_i}}{\sum_{n=1}^{N} e^{z_n}} = e^{z_j}$ 

# Math Addendum - Vector Calculus



Let us start with a column vector $\vec{v}$ that is of length x. $\vec{v}$ is the product of matrix $M$ of x rows and y columns with column vector $\vec{c}$  of length y. 

$$\vec{v} = M \vec{y}$$

Alright we have our column vector $\vec{v}$ now. Next let's explore how to take the derivative of $\vec{v}$ with respect to $\vec{y}$. Doing this fully would involve x * y derivatives since $\vec{v}$ is length x and $\vec{y}$ is length y. To start simply, let's look at just the derivative of the first number in $\vec{v}$ with respect to the second number in $\vec{y}$.

$$ \frac{\partial{\vec{v}_1}}{\partial{\vec{y}_2}} $$

So the first step is look at the equation that formed $\vec{v}_1$. This involved multiplying the first row of $W$ with each member of $\vec{y}$.

$$ \vec{v}_1 = \sum_{n=1}^{y} M_{1,n} \ \vec{y}_n$$

We can break this equation down a bit more and remove the sum for easier processing.

$$ \vec{v}_1 = M_{1,1} \ \vec{y}_1 + M_{1,2}\ \vec{y}_2 $$

Looking at it in this form, it is easy to quickly see that most of the terms do not matter when we take the partial (this is very useful when there are many other terms).
$$ 
\frac{\partial{\vec{v}_1}}{\partial{\vec{y}_2}} = \frac{\partial{}}{\partial{\vec{y}_2}}  M_{1,1} \ \vec{y}_1 + M_{1,2}\ \vec{y}_2 \\
\\
= \ 0 + \frac{\partial{}}{\partial{\vec{y}_2}} M_{1,2}\ \vec{y}_2 \\
\\
= M_{1,2}
$$

This process turns out to be true for all partials of of $\partial{v}$ with respect ot $\partial{y}$. Thus, we can now form a matrix with all of our partials in it which was our original goal.
$$ 
\frac{\partial{\vec{v}}}{\partial{\vec{y}}} = 
\begin{bmatrix}
    \frac{\partial{\vec{v}_1}}{\partial{\vec{y}_1}}       &  \frac{\partial{\vec{v}_1}}{\partial{\vec{y}_2}} \\
    \frac{\partial{\vec{v}_2}}{\partial{\vec{y}_1}}       &  \frac{\partial{\vec{v}_2}}{\partial{\vec{y}_2}} \\
    \frac{\partial{\vec{v}_3}}{\partial{\vec{y}_1}}       &  \frac{\partial{\vec{v}_3}}{\partial{\vec{y}_2}} \\
\end{bmatrix}
=
\begin{bmatrix}
   M_{1,1} & M_{1,2} \\
   M_{2,1} & M_{2,2} \\
   M_{3,1} & M_{3,2}
\end{bmatrix}
= M
$$


# Sources

Thanks to the following which helped with this post:

http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/    Neural Network in Python  
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/   Softmax derivative math  
http://colah.github.io/posts/2015-08-Backprop/   Reverse Auto-diff help
